## Chat Bot.

In this work we are going to create a simple framework to create chat bots.

### First of all we need to define the intents file that looks like the following.
That file consist of the follwong parts:
1. Tag
2. Patterns
3. Responses
4. Context we set

In [1]:
import json

with open('intents.json') as data:
    intents = json.load(data)

![Intents example:](img/image.png)

Using intents file we are going to create the following objects:
1. Words
2. Classes
3. Documents

In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer

def get_objects(intents):
    words = []
    classes = []
    documents = []
    ignore_words = ['?']
    stemmer = LancasterStemmer()
    # loop through each sentence in our intents patterns
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            # tokenize each word in the sentence
            w = nltk.word_tokenize(pattern)
            # add to our words list
            words.extend(w)
            # add to documents in our corpus
            documents.append((w, intent['tag']))
            # add to our classes list
            if intent['tag'] not in classes:
                classes.append(intent['tag'])
                
    # stem and lower each word and remove duplicates
    words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
    words = sorted(list(set(words)))
    
    # remove duplicates
    classes = sorted(list(set(classes)))

    print (len(documents), "documents")
    print (len(classes), "classes", classes)
    print (len(words), "unique stemmed words", words)
    
    return words, classes, documents, stemmer

Now we need to prepare data for training:

In [3]:
import numpy as np
import tensorflow as tf
import tflearn
import random

def get_data(words, classes, documents, stemmer):
    # create our training data
    training = []
    output = []
    # create an empty array for our output
    output_empty = [0] * len(classes)

    # training set, bag of words for each sentence
    for doc in documents:
        # initialize our bag of words
        bag = []
        # list of tokenized words for the pattern
        pattern_words = doc[0]
        # stem each word
        pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
        # create our bag of words array
        for w in words:
            bag.append(1) if w in pattern_words else bag.append(0)

        # output is a '0' for each tag and '1' for current tag
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1

        training.append([bag, output_row])

    # shuffle our features and turn into np.array
    random.shuffle(training)
    training = np.array(training)

    # create train and test lists
    train_x = list(training[:,0])
    train_y = list(training[:,1])
    
    return train_x, train_y

Let's build a model:

In [12]:
def build_model(train_x, train_y):
    tf.reset_default_graph()

    net = tflearn.input_data(shape=[None, len(train_x[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
    net = tflearn.regression(net)

    model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
    model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
    model.save('model.tflearn')
    return model

Let's define the function that splts and tokenizes sentence:

In [5]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

Let's define function that returns a bag of words for a given sentence:

In [6]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

## 1. Get words, classes and  documents objects:

In [7]:
words, classes, documents, stemmer = get_objects(intents)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


## 2. Get training data:

In [8]:
train_x, train_y = get_data(words, classes, documents, stemmer)

In [9]:
np.array(train_y).shape

(27, 9)

## 3. Build and train a model:

In [13]:
model = build_model(train_x, train_y)

Training Step: 3999  | total loss: 0.52720 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.52720 - acc: 0.9332 -- iter: 24/27
Training Step: 4000  | total loss: 0.49757 | time: 0.022s
| Adam | epoch: 1000 | loss: 0.49757 - acc: 0.9399 -- iter: 27/27
--
INFO:tensorflow:/home/oem/Projects/Chat_Bot/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## 4. Now we are able to get a prediction for a given sentence:

In [17]:
p = bow("is your shop open today?", words)
print(model.predict([p]))

[[7.6470144e-02 8.9529996e-05 1.0295273e-02 6.5508998e-06 7.1767247e-01
  8.5564163e-05 3.1676062e-03 9.7906508e-04 1.9123383e-01]]


### Also we can get a tag:

In [19]:
pred_tag = np.argmax((model.predict([p])))
print(classes[pred_tag])

opentoday


Let's define a function that will give us a response:

In [20]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

### Let's test it now:

#### 1. Classification of a sentence:

In [21]:
classify('is your shop open today?')

[('opentoday', 0.71767247)]

#### 2. Get a response to a message:

In [22]:
response('is your shop open today?')

Our hours are 9am-9pm every day


In [23]:
response('do you take cash?')

We accept VISA, Mastercard and AMEX


In [24]:
response('what kind of mopeds do you rent?')

We have Piaggio, Vespa and Yamaha mopeds


In [25]:
response('Goodbye, see you later')

See you later, thanks for visiting


#### 3. Let's check a context after this messages:

In [26]:
context

{}

#### 4. Let's check that context is working:

In [27]:
response('we want to rent a moped')

Are you looking to rent today or later this week?


In [28]:
context

{'123': 'rentalday'}

In [29]:
response('today')

For rentals today please call 1-800-MYMOPED
